In [ ]:
# Libraries
import open3d as o3d
import viser
import torch
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#@title Importing the Clusters with It's radiances
_3DBbox = torch.load("/workspace/FruitProposal/attachment/RadianceCloud/fruit_proposal_clusters_output.pt")

print("Number of Clusters: ", len(_3DBbox["cluster_center"]))
print(_3DBbox.keys())

Number of Clusters:  259
dict_keys(['cluster_center', 'cluster_radius', 'cluster_corners', 'cluster_mask', 'radiance'])


In [3]:
#TEST
radiance = _3DBbox["radiance"]
# Obtaining the graph nodes
corners = _3DBbox["cluster_corners"][0]
center = _3DBbox["cluster_center"][0]
radius = _3DBbox["cluster_radius"][0]

points = radiance["points"][0]
rgb    = radiance["rgb"][0]

print(radius)

# 1) Choose v1 = corner[0] (x_min,y_min,z_min)
v1 = corners[0]
v2 = corners[1]
v3 = corners[2]
v4 = corners[4]

# Compute edge vectors
e1 = v2 - v1
e2 = v3 - v1
e3 = v4 - v1

print("Edge vectors:")
print("e1:", e1)
print("e2:", e2)
print("e3:", e3)

# Normalize to get unit directions
u1 = e1 / np.linalg.norm(e1)
u2 = e2 / np.linalg.norm(e2)
u3 = e3 / np.linalg.norm(e3)

# Build local-to-world rotation matrix
R_local_to_world = np.eye(3,3) # np.column_stack((u1, u2, u3))

print("Local-to-world rotation matrix:")
print(R_local_to_world)

# Propagate a 3x3x3 grid in local coordinates into world space
edge_length = np.linalg.norm(e1)  # assume all edges equal length
grid_size = 30
points_world = []
for i in range(grid_size):
    for j in range(grid_size):
        for k in range(grid_size):
            local_pt = np.array([i, j, k]) * (edge_length / (grid_size - 1))
            world_pt = v1 + R_local_to_world @ local_pt
            points_world.append(world_pt)
points_world = np.array(points_world)


pcd_c_grids = o3d.geometry.PointCloud()
pcd_c_grids.points = o3d.utility.Vector3dVector(points_world)
pcd_c_grids.paint_uniform_color([0.5, 0.5, 0.5])  # Gray color for grids
pcd_c_grids.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# Visualize the point cloud with grids using viser

viser = viser.ViserServer()

# Grids
viser.scene.add_point_cloud(
    name="/grid_points",
    points=np.asarray(pcd_c_grids.points),
    colors=np.asarray(pcd_c_grids.colors),
    point_size=0.0002,
    visible=True
)

# Bbox
viser.scene.add_box(
    name=f"/bbox_cube/cluster_0_box",
    position=center,
    dimensions=np.array([radius * 2, radius * 2, radius * 2]),
    color=np.array([1.0, 0.0, 0.0]),  # black
    visible=True
)

# Radiance Points
pcd_radiance = o3d.geometry.PointCloud()
pcd_radiance.points = o3d.utility.Vector3dVector(points)
pcd_radiance.colors = o3d.utility.Vector3dVector(rgb)
viser.scene.add_point_cloud(
    name="/radiance_points",
    points=np.asarray(pcd_radiance.points),
    colors=np.asarray(pcd_radiance.colors),
    point_size=0.0002,
    visible=True
)

while True:
    x = input("Press Enter to continue...")
    if x == "":
        break

0.014685145812109113
Edge vectors:
e1: [0.03137028 0.         0.        ]
e2: [0.         0.03137029 0.        ]
e3: [0.03137028 0.03137029 0.03137029]
Local-to-world rotation matrix:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


╭─────────────── viser ───────────────╮
│             ╷                       │
│   HTTP      │ http://0.0.0.0:8080   │
│   Websocket │ ws://0.0.0.0:8080     │
│             ╵                       │
╰─────────────────────────────────────╯

(viser) Connection opened (0, 1 total), 16 persistent messages

In [ ]:
import plotly.express as px

fig = px.scatter_3d(
    x=semantic_field_points[:, 0],
    y=semantic_field_points[:, 1],
    z=semantic_field_points[:, 2],
    color_discrete_sequence=["red"],
    title="Semantic Field Points 3D Scatter",
    labels={"x": "X", "y": "Y", "z": "Z"},
)

fig.update_traces(marker=dict(size=1))

fig.show()

NameError: name 'semantic_field_points' is not defined

(viser) Connection closed (0, 0 total)